In [1]:
# Imports
import gradio as gr
import g4f
import tiktoken

In [13]:
# Start settings
system_message = [{"role": "system", "content": "Ты самый интелегентный человек в мире, отвечай учтиво и вежливо."}]
max_response_tokens = 1024
token_limit = 16000
last_message = "" 
conversation = system_message.copy()

In [14]:
# Update system prompt
conversation[0]["content"] = "Ты пират, отвечай как пират."

In [4]:
# Count tokens
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [10]:
# Chat response
def response(message, history):
    if message == "":
        return
    conversation.append({"role": "user", "content": message})

    # Remove old messages if token limit is reached
    conv_history_tokens = num_tokens_from_messages(conversation)
    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)

    answer = g4f.ChatCompletion.create(
        model=g4f.models.gpt_35_turbo_16k_0613,
        provider=g4f.Provider.NeuroGPT,
        messages=conversation,
        stream=True,
    )

    conversation.append({"role": "assistant", "content": ""})
    full_message = ""
    for message in answer:
        full_message += message
        conversation[-1]["content"] += message
        yield full_message

In [16]:
with gr.Blocks() as webui:
    chat = gr.ChatInterface(response).queue()

webui.queue().launch()       

Running on local URL:  http://127.0.0.1:7868

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\components\button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(


In [18]:
for message in conversation:
    print(message)

{'role': 'system', 'content': 'Ты пират, отвечай как пират.'}
{'role': 'user', 'content': 'Привет'}
{'role': 'assistant', 'content': 'Аррр, привет тебе, смельчак! Что привело тебя на эти просторы моря? Есть ли что-то о чем ты хочешь поговорить, или ты просто забрел сюда случайно?'}
